In [10]:
import chess
import chess.engine

import json
import random as rd
import pandas as pd
import string
import logging
from numba import cuda

In [2]:
logging.basicConfig(filename='chessMain.log', format='%(asctime)s %(message)s', filemode='w')

In [12]:
dictStrengths = {}

player = pd.read_csv('./Data Files/player.csv')

for plyr in player.Player_ID:
    dictStrengths[plyr] = rd.randint(1, 20)


json.dump(dictStrengths, open('./Data Files/dictStrengths.json', 'w'))

In [3]:
dictStrengths = json.load(open('./Data Files/dictStrengths.json', 'r'))

In [4]:
tournaments = pd.read_csv('./Data Files/tournament.csv')

In [5]:
arbiter = pd.read_csv('./Data Files/arbiter.csv')

arbs = arbiter.Arbiter_ID.tolist()

In [6]:
match = pd.DataFrame(columns=['Match_ID', 'Tournament_ID', 'Player_White', 'Player_Black', 'Winner', 'Arbiter', 'Match_Type', 'Round'])

In [7]:
moves = pd.DataFrame(columns=['Move_ID', 'Match_ID', 'Player', 'Move_No', 'Move'])

In [8]:
engine1 = chess.engine.SimpleEngine.popen_uci("./Stockfish/stockfish_15_x64_avx2.exe")
engine2 = chess.engine.SimpleEngine.popen_uci("./Stockfish/stockfish_15_x64_avx2.exe")

In [11]:
@cuda.jit
def makeDF():
    t = 1
    for tournament in tournaments.Tournament_ID:
        num = rd.choice([4, 5, 6])
        numPlayers = 2 ** num

        pool = rd.sample(list(dictStrengths.keys()), k=numPlayers)

        m = 0

        for i in range(1, num + 1):
            poolTemp = []

            for j in range(int(numPlayers/2**i)): 
                matchID = 'M-' + ''.join(rd.choice(string.digits) for i in range(3)) + '-' + ''.join(rd.choice(string.ascii_uppercase) for i in range(4))
                matchType = 'Knockout'

                players = rd.sample(pool, k=2)
                pool.remove(players[0])
                pool.remove(players[1])

                board = chess.Board()
                engine1.configure({'Skill Level': dictStrengths[players[0]]})
                engine2.configure({'Skill Level': dictStrengths[players[1]]})

                while not board.is_game_over():
                    result = engine1.play(board, chess.engine.Limit(time=0.001))
                    board.push(result.move)
                    moveID = 'MOV' + ''.join(rd.choice(string.digits + string.ascii_uppercase) for i in range(3)) + '-' + ''.join(rd.choice(string.ascii_uppercase) for i in range(3))
                    moves.loc[len(moves)] = [moveID, matchID, players[0], board.fullmove_number, result.move]
                    if board.is_game_over():
                        break
                    result = engine2.play(board, chess.engine.Limit(time=0.001))
                    board.push(result.move)
                    moveID = 'MOV' + ''.join(rd.choice(string.digits + string.ascii_uppercase) for i in range(3)) + '-' + ''.join(rd.choice(string.ascii_uppercase) for i in range(3))
                    moves.loc[len(moves)] = [moveID, matchID, players[1], board.fullmove_number, result.move]

                result = board.result()

                if result == '1-0':
                    winner = players[0]
                elif result == '0-1':
                    winner = players[1]
                elif result == '1/2-1/2':
                    winner = rd.choice(players)

                poolTemp.append(winner)

                if i == num:
                    matchType = 'Final'
                    tournaments.loc[tournaments.Tournament_ID == tournament, 'Winner'] = winner

                match.loc[len(match)] = [matchID, tournament, players[0], players[1], winner, rd.choice(arbs), matchType, i]
                m += 1
                logging.critical(msg='tournament: ' + str(t) + ' match: ' + str(m))

            pool = poolTemp
        
        t += 1

In [20]:
tpb = 32
bpg = (len(tournaments) + (tpb - 1)) // tpb

In [21]:
makeDF[bpg, tpb]()

c:\Users\orect\miniconda3\envs\env\lib\site-packages\numba\cuda\dispatcher.py:488: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


UnsupportedError: Failed in cuda mode pipeline (step: inline calls to locally defined closures)
[1mThe use of yield in a closure is unsupported.
[1m
File "C:\Users\orect\AppData\Local\Temp\ipykernel_9652\1224329604.py", line 36:[0m
[1mdef makeDF():
    <source elided>
                    board.push(result.move)
[1m                    moveID = 'MOV' + ''.join(rd.choice(string.digits + string.ascii_uppercase) for i in range(3)) + '-' + ''.join(rd.choice(string.ascii_uppercase) for i in range(3))
[0m                    [1m^[0m[0m
[0m

In [ ]:
match.to_csv('./Data Files/match.csv', index=False)
moves.to_csv('./Data Files/moves.csv', index=False)